In [6]:
from pymongo import MongoClient
import pandas
from tqdm import tqdm_notebook as tqdm
import re
client = MongoClient()
db = client.get_database('wikipedia')

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from time import time

n_samples = len(content)
n_features = 1000
n_components = 64
n_top_words = 50

In [8]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [79]:
print("Loading dataset...")
t0 = time()
content = []
pattern = re.compile('[a-zA-Z ,\.]+')
titles=[]
i=0
for page in tqdm(db.page_text.find()):
    i+=1
    text=[]
    for t in re.findall(pattern, page['text']):
        if len(t.split(' '))<3:
            continue
        text.append(t)
    content.append(' '.join(text))
    titles.append(page['title'])
print("done in %0.3fs." % (time() - t0))

Loading dataset...


Exception in thread Thread-46:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/xuanli2/penv/lib/python3.5/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/xuanli2/penv/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




done in 123.203s.


In [108]:
sample=content[:n_samples]

In [109]:
len(sample)

76853

In [110]:
from sklearn.feature_extraction import stop_words
 
stopwords=list(stop_words.ENGLISH_STOP_WORDS)

In [111]:
stopwords+=['ref','http','www','com','org','html','jpg','REDIRECT']

In [ ]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=stopwords)
t0 = time()
tf = tf_vectorizer.fit_transform(sample)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...


In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,
                                n_jobs=4,
                               )
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

In [119]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: language english used word languages words use latin term names dictionary meaning form letter written common example old like modern forms called writing person different th standard usually native greek origin means similar known related sound middle uses text official list terms century commonly given include international based present version
Topic #1: american player english singer actor author producer president director united born writer italian minister general states british baseball war french people john coach day football prime manager russian year founded south indian spanish ii william new race battle artist irish model ice king car world german television african leader date
Topic #2: john david james paul michael smith peter robert jones jack mark jr george william thomas richard usa lee brown martin new mary williams van charles day man hill st end hall fox mr ed book ii york don company house time edward directed born henry list white

In [ ]:
import pickle

In [ ]:
pickle.dump([titles,lda,tf],open('./data/dump/lda.pkl','wb'))

In [120]:
result=lda.transform(tf)

In [121]:
result.shape

(76853, 64)

In [127]:
titles[0]

'List_of_sex_positions'

In [128]:
with open('./data/dump/lda_embeddint.txt','w') as fout:
    fout.write('%d %d\n'%(n_samples,n_components))
    for i in tqdm(range(n_samples)):
        if ' ' in str(titles[i]):
            print(i)
            continue
        d=' '.join(map(lambda x:str(x), result[i]))
        fout.write('%s %s\n'%(str(titles[i]),d))